### look_up_ADCIRC_tide

Read in a `.mat` file produced by `predict_tide_capecod.m`, as demo'd in `test_tide.m`.  
Interpolate tides for input time.  

The data was written from Matlab like this:  
`save('HoM_ADCIRC_tide_predictions.mat','lat','lon','T','tid','-v7.3')`  
The flag `-v7.3` ensures hdf5 format.

The interpolation is done using Matlab datenum values as the time axis, so a function to change datetime objects  
to datenum values is included.

In [1]:
import numpy as np
import datetime as dt
from scipy.interpolate import interp1d
import h5py

matfile = 'HoM_ADCIRC_tide_predictions.mat'

#### Function that reads tidal dat in `.mat` format and creates an interpolation function object

In [2]:
def get_interp_function(matfile):
    """Read tidal data in .mat format, create interpolation object
    """
    data = h5py.File(matfile,'r')
    T = np.squeeze(np.array(data['T']))
    tid = np.squeeze(np.array(data['tid']))
    return interp1d(T, tid, kind='linear')

#### Function to convert datetime ojbect to Matlab datenum

In [3]:
def datetime2matlabdn(date):
    """Convert datetime object to Matlab datenum
    https://stackoverflow.com/questions/8776414/python-datetime-to-matlab-datenum
    """
    ord = date.toordinal()
    mdn = date + dt.timedelta(days = 366)
    frac = (date-dt.datetime(date.year,date.month,date.day,0,0,0)).seconds / (24.0 * 60.0 * 60.0)
    return mdn.toordinal() + frac

#### Call `get_interp_function` to get an interpolation function object

In [4]:
f = get_interp_function(matfile)

#### Make an array of datetime times to test

In [5]:
t = np.arange(dt.datetime(2020,10,1,13,30), dt.datetime(2022,7,1,13,30), dt.timedelta(days=1)).astype(dt.datetime)

#### Test the numpy interpolation

In [6]:
%%time
for time in t:
    dn = datetime2matlabdn(time)
    tidi = f(dn)

Wall time: 15 ms


#### Move tide data from the hdf5 dataset to 1D numpy arrays

In [7]:
data = h5py.File(matfile,'r')
T = np.squeeze(np.array(data['T']))
tid = np.squeeze(np.array(data['tid']))

#### Test the scipy interpolation (way faster)

In [8]:
%%time
for time in t:
    dn = datetime2matlabdn(time)
    tidi = np.interp(dn, T, tid)

Wall time: 5.98 ms
